In [14]:
%load_ext autoreload
%autoreload 2

import function_list
from pimp import *

import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_log_error

train = pd.read_csv('train.csv', parse_dates=['date'])
test = pd.read_csv('test.csv', parse_dates=['date'])
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv", parse_dates=["date"])
oil = pd.read_csv("oil.csv", parse_dates=["date"])
holidays = pd.read_csv("holidays_events.csv", parse_dates=["date"])

# ==================
# データの結合
# ==================
df = pd.concat([train, test], axis=0, sort=False)

# [変更点 1] category化を「merge前の仮」ではなく、merge後に確定させるためここでは残してもよいが、後で再度確定する
cat_cols = ["store_nbr", "family"]
for c in cat_cols:
    df[c] = df[c].astype("category")

df = df.merge(stores, on='store_nbr', how='left', validate="m:1")
df = df.merge(transactions, on=['date', 'store_nbr'], how='left', validate="m:1")
df = df.merge(oil, on='date', how='left', validate="m:1")

# 祝日を日単位で集約
holidays_day = holidays.groupby("date").size().reset_index(name="n_holidays")
df = df.merge(holidays_day, on="date", how="left", validate="m:1")
df['n_holidays'] = df['n_holidays'].fillna(0).astype(int)

# [変更点 2] mergeでobjectに戻ることがあるので、merge後にcategoryを「確定」させる（エラー予防）
for c in cat_cols:
    df[c] = df[c].astype("category")

# [変更点 3] transactions/oil の欠損を最低限埋める（LB不安定化の予防）
if "transactions" in df.columns:
    df["transactions"] = df["transactions"].fillna(0)

# oil列名は環境で違うので存在する数値列を前方補完（dcoilwtico等を想定）
oil_cols = [c for c in oil.columns if c != "date"]
for c in oil_cols:
    if c in df.columns:
        df[c] = df[c].ffill()

# ==================
# 特徴量作成
# ==================
df['weekday'] = df['date'].dt.weekday
df['month'] = df['date'].dt.month

# [変更点 4] lag作成の前に必ず時系列順にソート（shiftは行順依存なので必須）
df = df.sort_values(["store_nbr", "family", "date"]).reset_index(drop=True)

df['lag_7'] = df.groupby(['store_nbr', 'family'])['sales'].shift(7)
df['lag_14'] = df.groupby(['store_nbr', 'family'])['sales'].shift(14)

df["roll_7"] = (
    df.groupby(["store_nbr", "family"])["sales"]
      .transform(lambda s: s.shift(1).rolling(7, min_periods=1).mean())
)
df["roll_14"] = (
    df.groupby(["store_nbr", "family"])["sales"]
      .transform(lambda s: s.shift(1).rolling(14, min_periods=1).mean())
)

# =========================
# trainとtestを分割する
# =========================
cutoff = test['date'].min()
train_df = df[df['date'] < cutoff].copy()
test_df  = df[df['date'] >= cutoff].copy()

# =========================
# 直前専用 CV（2017/08/01-15）
# =========================
valid = train_df[
    (train_df["date"] >= "2017-08-01") &
    (train_df["date"] <  "2017-08-16")
].copy()

train_cv = train_df[train_df["date"] < "2017-08-01"].copy()

features = ["store_nbr", "family", "weekday", "month", "lag_7", "lag_14", "roll_7", "roll_14"]
cat_features = ["store_nbr", "family"]

# =========================
# RMSLE 評価関数（salesスケール）: sklearn API用 (y_true, y_pred)
# =========================
def rmsle_eval(y_true, y_pred):
    y_pred = np.maximum(y_pred, 0)
    val = np.sqrt(mean_squared_log_error(y_true, y_pred))
    return "rmsle", float(val), False

# =========================
# Optuna（単目的）
# =========================
def objective(trial):
    optuna_params = {
        "objective": "regression",
        "learning_rate": 0.05,
        "num_leaves": trial.suggest_int("num_leaves", 32, 128),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.7, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "n_estimators": 600,
        "n_jobs": -1,
        "seed": 42,
    }

    model = lgb.LGBMRegressor(**optuna_params)
    model.fit(
        train_cv[features],
        train_cv["sales"],
        categorical_feature=cat_features,
        eval_set=[(valid[features], valid["sales"])],
        eval_metric=rmsle_eval,
        callbacks=[lgb.early_stopping(50, verbose=False)],
    )

    # [変更点 5] best_iteration_ を後段で使えるように保存
    trial.set_user_attr("best_iteration", int(model.best_iteration_))

    pred = model.predict(valid[features])
    score = np.sqrt(mean_squared_log_error(valid["sales"], np.maximum(pred, 0)))
    return float(score)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("BEST RMSLE:", study.best_value)
print("BEST PARAMS:", study.best_params)

# =========================
# 最終モデル（全 train で再学習）
# =========================
# [変更点 6] best_iteration が保存されている前提で、最終のn_estimatorsに反映
best_iter = int(study.best_trial.user_attrs.get("best_iteration", 600))

main_params = {
    **study.best_params,
    "objective": "regression",
    "learning_rate": 0.05,
    "n_estimators": best_iter,
    "n_jobs": -1,
    "seed": 42,
}

main_model = lgb.LGBMRegressor(**main_params)
main_model.fit(
    train_df[features],
    train_df["sales"],
    categorical_feature=cat_features
)

# =========================
# 予測（id順厳守）
# =========================
test_df = test_df.sort_values("id").reset_index(drop=True)

pred = main_model.predict(test_df[features])
pred = np.maximum(pred, 0)

submission = pd.DataFrame({
    "id": test_df["id"],
    "sales": pred
})

submission.to_csv("submission.csv", index=False)
print("submission.csv saved")


C:\Users\ohtak\AppData\Local\Temp\ipykernel_3832\178293690.py:56: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['lag_7'] = df.groupby(['store_nbr', 'family'])['sales'].shift(7)
C:\Users\ohtak\AppData\Local\Temp\ipykernel_3832\178293690.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['lag_14'] = df.groupby(['store_nbr', 'family'])['sales'].shift(14)
C:\Users\ohtak\AppData\Local\Temp\ipykernel_3832\178293690.py:60: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to ado

[LightGBM] [Warning] feature_fraction is set=0.7368401117089458, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7368401117089458
[LightGBM] [Warning] bagging_fraction is set=0.7154528963079622, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7154528963079622
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7368401117089458, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7368401117089458
[LightGBM] [Warning] bagging_fraction is set=0.7154528963079622, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7154528963079622
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018535 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

[I 2025-12-17 03:21:54,811] Trial 0 finished with value: 0.4693177638706481 and parameters: {'num_leaves': 126, 'min_child_samples': 132, 'feature_fraction': 0.7368401117089458, 'bagging_fraction': 0.7154528963079622, 'bagging_freq': 2}. Best is trial 0 with value: 0.4693177638706481.


[LightGBM] [Warning] feature_fraction is set=0.7368401117089458, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7368401117089458
[LightGBM] [Warning] bagging_fraction is set=0.7154528963079622, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7154528963079622
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.734473907895152, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.734473907895152
[LightGBM] [Warning] bagging_fraction is set=0.867575865674876, subsample=1.0 will be ignored. Current value: bagging_fraction=0.867575865674876
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.734473907895152, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.734473907895152
[LightGBM] [Warning] bagging_fract

[I 2025-12-17 03:22:07,695] Trial 1 finished with value: 0.48608922237467506 and parameters: {'num_leaves': 93, 'min_child_samples': 36, 'feature_fraction': 0.734473907895152, 'bagging_fraction': 0.867575865674876, 'bagging_freq': 6}. Best is trial 0 with value: 0.4693177638706481.


...

[LightGBM] [Warning] feature_fraction is set=0.8683860939652829, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8683860939652829
[LightGBM] [Warning] bagging_fraction is set=0.8469057088360546, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8469057088360546
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8877283268665155, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8877283268665155
[LightGBM] [Warning] bagging_fraction is set=0.9986359904304885, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9986359904304885
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8877283268665155, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8877283268665155
[LightGBM] [Warning] bagging

[I 2025-12-17 03:27:44,646] Trial 18 finished with value: 0.4540816294945279 and parameters: {'num_leaves': 106, 'min_child_samples': 130, 'feature_fraction': 0.8877283268665155, 'bagging_fraction': 0.9986359904304885, 'bagging_freq': 3}. Best is trial 8 with value: 0.4459250923411441.


[LightGBM] [Warning] feature_fraction is set=0.8877283268665155, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8877283268665155
[LightGBM] [Warning] bagging_fraction is set=0.9986359904304885, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9986359904304885
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9605451610755226, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9605451610755226
[LightGBM] [Warning] bagging_fraction is set=0.9252004677319888, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9252004677319888
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.9605451610755226, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9605451610755226
[LightGBM] [Warning] bagging

[I 2025-12-17 03:28:05,661] Trial 19 finished with value: 0.45199228773567385 and parameters: {'num_leaves': 89, 'min_child_samples': 149, 'feature_fraction': 0.9605451610755226, 'bagging_fraction': 0.9252004677319888, 'bagging_freq': 7}. Best is trial 8 with value: 0.4459250923411441.


[LightGBM] [Warning] feature_fraction is set=0.9605451610755226, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9605451610755226
[LightGBM] [Warning] bagging_fraction is set=0.9252004677319888, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9252004677319888
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
BEST RMSLE: 0.4459250923411441
BEST PARAMS: {'num_leaves': 104, 'min_child_samples': 109, 'feature_fraction': 0.9552119708346797, 'bagging_fraction': 0.973335460554759, 'bagging_freq': 3}
[LightGBM] [Warning] feature_fraction is set=0.9552119708346797, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9552119708346797
[LightGBM] [Warning] bagging_fraction is set=0.973335460554759, subsample=1.0 will be ignored. Current value: bagging_fraction=0.973335460554759
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_fre

特徴量を作る